## SKT-GPT2


In [ ]:
!pip install transformers

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

In [ ]:
from transformers import AutoConfig
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', pad_token = '<PAD>')

config = AutoConfig.from_pretrained('skt/kogpt2-base-v2')
config
# model = TFAutoModel.from_config(config)

In [ ]:
config.update_from_string("vocab_size=52002")

In [ ]:
model = TFGPT2LMHeadModel.from_config(config)

In [ ]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)
print('-' * 10)
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))

51200
51200
51201
----------
</s>
<usr>
<pad>
<sys>


In [ ]:
tokenizer.all_special_tokens

['<|endoftext|>', '<PAD>']

In [ ]:
import pandas as pd
import tqdm
import urllib.request

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')

In [ ]:
train_data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [ ]:
len(train_data)

11823

In [ ]:
batch_size = 32

In [ ]:
def get_chat_data():
  for question, answer in zip(train_data.Q.to_list(), train_data.A.to_list()):
    bos_token = [tokenizer.bos_token_id]
    eos_token = [tokenizer.eos_token_id]
    sent = tokenizer.encode('<usr>' + question +'<sys>'+ answer) 
    yield bos_token + sent + eos_token
     

In [ ]:
dataset = tf.data.Dataset.from_generator(get_chat_data, output_types=tf.int32)

In [ ]:
dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)

In [ ]:
for batch in dataset:
    print(batch)
    break     

In [ ]:
tokenizer.decode(batch[0])

'<|endoftext|><usr> 12시 땡!<sys> 하루가 또 가네요.<|endoftext|><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>'

In [ ]:
print(tokenizer.encode(' 12시 땡! 하루가 또 가네요.'))

[9349, 7888, 739, 7318, 376, 12557, 6824, 9108, 9028, 7098, 25856]


In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

In [ ]:

steps = len(train_data) // batch_size + 1
print(steps)


370


In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
  epoch_loss = 0

  for batch in tqdm.tqdm_notebook(dataset, total=steps):
      with tf.GradientTape() as tape:
          result = model(batch, labels=batch)
          loss = result[0]
          batch_loss = tf.reduce_mean(loss)
          
      grads = tape.gradient(batch_loss, model.trainable_variables)
      adam.apply_gradients(zip(grads, model.trainable_variables))
      epoch_loss += batch_loss / steps

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

<ipython-input-70-b887b383bcd0>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm.tqdm_notebook(dataset, total=steps):


  0%|          | 0/370 [00:00<?, ?it/s]

[Epoch:    1] cost = 2.89349723


  0%|          | 0/370 [00:00<?, ?it/s]

[Epoch:    2] cost = 2.65099168


...

[Epoch:    3] cost = 2.42602205


  0%|          | 0/370 [00:00<?, ?it/s]

[Epoch:    4] cost = 2.21267653


  0%|          | 0/370 [00:00<?, ?it/s]

[Epoch:    5] cost = 2.0102973


In [ ]:
text = '오늘도 좋은 하루!'
sent = '<usr>' + text + '<sys>'

In [ ]:
input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])    

In [ ]:
output = model.generate(input_ids, max_length=50, early_stopping=True, eos_token_id=tokenizer.eos_token_id)
output

<tf.Tensor: shape=(1, 16), dtype=int32, numpy=
array([[51200,     2, 10070,  7235, 10586, 12557,   376,     4,  9022,
         6866,  9622,  8137,  6853, 27511, 25856, 51200]], dtype=int32)>

In [ ]:
decoded_sentence = tokenizer.decode(output[0].numpy().tolist())

In [ ]:
decoded_sentence.split('<sys> ')[1].replace("<|endoftext|>", "")

'그게 맞을거 같아요.'

In [ ]:

def return_answer_by_chatbot(user_text):
  sent = '<usr>' + user_text + '<sys>'
  input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
  input_ids = tf.convert_to_tensor([input_ids])
  output = model.generate(input_ids, max_length=50, do_sample=True, top_k=20)
  sentence = tokenizer.decode(output[0].numpy().tolist())
  chatbot_response = sentence.split('<sys> ')[1].replace("<|endoftext|>", "").strip("<PAD>")
  return chatbot_response

In [ ]:
return_answer_by_chatbot('안녕! 반가워~')


'서로에게 물어보는게 좋을 것 같아요.'

In [ ]:
return_answer_by_chatbot('너는 누구야?')

In [ ]:
return_answer_by_chatbot('나랑 영화보자')